# __텍스트로 텍스트 검색하기__

- 튜토리얼 난이도: ★★☆☆☆
- 읽는데 걸리는 시간: 7분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_search/search_text_by_text.ipynb
- 참고 문서: [Naver sentiment movie corpus v1.0](https://github.com/e9t/nsmc#naver-sentiment-movie-corpus-v10), [단어 임베딩: 어휘의 의미(LEXICAL SEMANTICS)를 인코딩하기](https://tutorials.pytorch.kr/beginner/nlp/word_embeddings_tutorial.html), [컴퓨터가 바라보는 문자](https://heung-bae-lee.github.io/2020/01/16/NLP_01/)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">텍스트 수치화 기술 이해하기</h4>
    <p>컴퓨터는 사람이 쓰는 언어(자연어)를 곧바로 입력으로 받을 수 없습니다. 때문에 자연어를 기계가 인식할 수 있는 수치 데이터로 변환하는 과정이 필요합니다. 자연어 처리분야에서 임베딩이란 이렇게 사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자형태인 vector로 바꾼 결과 혹은 그 일련의 과정 전체를 의미합니다.</p>
</div>

자연어의 임베딩을 구하는 기법은 크게 통계적인 기법과, 인공신경망 기반 기법으로 구분되고, 세부적으로도 더 나눌 수 있습니다. ThanoSQL에서는 인공신경망 기반의 [자가학습모델(Self-Supervised Learning Model)](https://ko.wikipedia.org/wiki/%EC%9E%90%EA%B8%B0_%EC%A7%80%EB%8F%84_%ED%95%99%EC%8A%B5)을 사용하여, 텍스트를 잘 표현하는 벡터를 스스로 학습하는 방법을 제공합니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 이번 튜토리얼에서는 영화 리뷰 데이터를 사용합니다. 데이터는 영화 리뷰 텍스트와, 라벨값으로 구성되어있지만, 자가학습법을 사용해 훈련하는 예제를 보여드리기 위해 라벨값은 사용하지 않습니다. 10,000개의 영화 리뷰 데이터를 학습하여, 입력 문장과 유사한 문장 찾기, 문장의 키워드 추출하기를 해보겠습니다.</p>
</div>

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/1.5/ko/getting_started/paas/workspace/lab/)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET nsmc_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY nsmc_train
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_train.csv'

Success


In [4]:
%%thanosql
COPY nsmc_test
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"if_exists": 동일 이름의 테이블이 존재하는 경우 처리하는 방법 설정. 오류 발생, 기존 테이블에 추가, 기존 테이블 대체 (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

텍스트 수치화 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __nsmc_train__ 테이블을 사용합니다. __nsmc_train__ 테이블은 NAVER Sentiment Movie Corpus 영화 리뷰 데이터 및 라벨 정보의 일부가 담겨 있는 테이블입니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM nsmc_train
LIMIT 5

,id,document,label
0,9465891,저퀄리티. 뭐하나 장점이 없음.,0
1,9792014,10점 만점에 1점 !!! 최악이네 하아...,0
2,3795348,누가 억지눈물 잘흘리나 자기네들끼리 대결하는 작품,0
3,1129393,신도 싫고 인간도 싫다,1
4,4678610,추억의 명작만화,1


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>nsmc_train</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>id: 데이터의 식별 번호</li>
        <li>document: 영화 리뷰 데이터</li>
        <li>label: 라벨값</li>
    </ul>
</div>

## __2. 텍스트 수치화 모델 생성__

이전 단계에서 확인한 __nsmc_train__ 테이블을 사용하여 텍스트 수치화 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>nsmc_text_search_model</strong>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 2 min)

In [6]:
%%thanosql
BUILD MODEL nsmc_text_search_model
USING SBERTKo
OPTIONS (
    text_col='document',
    overwrite=True
    )
AS
SELECT *
FROM nsmc_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>nsmc_text_search_model</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>SBERTKo</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"max_epochs": 텍스트 수치화 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, optional, default: 1)</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

다음 "__CONVERT USING__ " 쿼리 구문을 실행하여 __nsmc_test__ 텍스트들을 수치화 합니다. 수치화 된 결과는 __nsmc_test__ 테이블에 사용자가 옵션으로 지정한 이름(default: 'convert_result')의 컬럼에 저장됩니다.

In [7]:
%%thanosql
CONVERT USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    batch_size=32,
    result_col='convert_result'
    )
AS
SELECT *
FROM nsmc_test

,id,document,label,convert_result
0,9181084,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[b'-', b'\xa6', b'\x9a', b'>', b'\xba', b'\x92..."
1,6318649,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[b'\xc6', b'i', b'\xad', b'\xbe', b'\xd3', b'\..."
2,5718332,유쾌하고 신나는 스피드 코미디.,1,"[b'\xcf', b'6', b'\xc0', b'\xbe', b'd', b'/', ..."
3,8063675,정말 따뜻했던 드라마ㅠㅠ,1,"[b'\xed', b'\x9d', b'\xad', b'>', b'\xc7', b'^..."
4,10229366,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[b'\xac', b'\xfb', b'\x0b', b'\xbb', b';', b'|..."
...,...,...,...,...
1995,5382227,30년간 본 수천편의 드라마중 최고는 이 작품,1,"[b'\xc0', b'-', b""'"", b'\xbe', b""'"", b'K', b'\..."
1996,4584773,빛돌을 이길 5명의 용사들의 패배,1,"[b'1', b'.', b'\xc9', b'\xbe', b'\x05', b'M', ..."
1997,2559484,빌머레이의 굳은 표정에 모든것이 살아있다.. 짐자무시 최고!,1,"[b'\xe3', b'\x9b', b'\xd3', b'<', b'\xc4', b'\..."
1998,9867081,이런영화에 투자하지 않는것이 진정한 한국영화의 발전을위한 초석,0,"[b'\xa9', b'\xdf', b'\x1c', b'>', b'@', b'~', ..."


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" 쿼리 구문은 <strong>nsmc_text_search_model</strong> 모델을 텍스트 수치화를 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"result_col": 데이터 테이블에서 수치화된 결과를 담을 컬럼 이름 (str, optional, default: 'convert_result')</li>
        </ul>
        </li>
    </ul>
</div>

다음 "__CONVERT USING__" 쿼리 구문을 실행하여 변환된 결과를 다른 __ThanoSQL__ 쿼리 구문들과 함께 활용할 수 있도록 새로운 테이블에 저장해봅시다.

In [8]:
%%thanosql
CREATE TABLE nsmc_test_convert AS 
SELECT * FROM ( 
    CONVERT USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        batch_size=32,
        result_col='convert_result'
        )
    AS
    SELECT *
    FROM nsmc_test
)

Success


## __3. 텍스트 수치화 모델을 사용하여 유사한 문서 검색__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서를 검색합니다.

In [9]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='영화 내용이 불만족스러웠다',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS 
    SELECT *
    FROM nsmc_test_convert
    )

,document,label,score
0,내가 생각했던 스토리와 조금 달랐지만 신선하고 독특했던 영화였다.,1,0.651934
1,너무 작위적인 스토리 전개..실망스러운 영화ㅠ,0,0.651376
2,잔잔한 영화 보기 불편할수도 있음,1,0.649874
3,정말 싱겁다 영화를 다봣어도 보다만 느낌이다,0,0.631995
4,왕건 후속편으로써 매우 실망스런 작품.,0,0.631769
5,"평점보고 기대해서 그런지 실망스런 영화 ,쏘쏘한 영화",0,0.628886
6,영화 전개가 너무 지루해요.,0,0.626460
7,배우들 영화 찍기 싫었을듯,0,0.625178
8,"부족한 감이 있었고, 암울했다.",0,0.622642
9,무관심했던 나를 반성케 한 영화,1,0.622506


In [10]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='기분이 좋아지는 작품',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS
    SELECT *
    FROM nsmc_test_convert
    )

,document,label,score
0,보고나면 기분 좋은 영화,1,0.783119
1,몰입도 잘되고 좋은 영화,1,0.715477
2,감동적인 영화,1,0.699444
3,너무 멋있는 영화. 환한 느낌을 준다,1,0.678274
4,정말 유쾌한 영화입니다!,1,0.677389
5,추억속의 영화 ㅎ 정말 재밌게 봤습니다!,1,0.675679
6,정말 따뜻했던 드라마ㅠㅠ,1,0.672637
7,좋아요,1,0.671109
8,멋진영화,1,0.663215
9,재밌다,1,0.661989


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 검색 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"search_by": 검색할 때 사용할 이미지|텍스트|오디오|비디오 타입 (str)</li>
            <li>"search_input": 검색할 때 사용할 입력값 (str)</li>
            <li>"emb_col": 데이터 테이블에서 수치화된 결과를 담은 컬럼 (str)</li>
            <li>"result_col": 데이터 테이블에서 검색 결과를 담을 컬럼 이름 (str, optional, default: 'search_result')</li>
            <li>"top_k": 반환할 행의 수. None을 입력할 시 데이터 테이블 전체를 반환 (int, optional, default: 1000)</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __4. 텍스트 수치화 모델을 사용하여 문서에서 키워드 추출__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서에서 키워드를 추출합니다.

In [11]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT *
FROM nsmc_test_convert
LIMIT 10 OFFSET 40

,id,document,label,convert_result,keyword
0,7928591,"일단 엠씨 역량 부족이 가장 큰 문제인듯. 시간 없다고, 재미없다고 게스트 말 끊는...",0,"[b'\xc3', b'\xe9', b'\x04', b'>', b'&', b'\r',...","{'keyword': ['안듣고 걍한마디하라고하고 끝ㅋㅋㅋ엠씨가노답임', '이야기하..."
1,8380896,방금 슬쩍 봤는데 '인삼' 처럼 생긴 '산삼' 암수가 서로 토닥이다가 주인공에게 잡...,0,"[b'\x16', b'\xff', b'\x0f', b'\xbf', b'\xf7', ...","{'keyword': ['주인공에게 잡힘 ㅋㅋ', '산삼 암수가 서로', '암수가 ..."
2,8914833,신세계에 비교하는 거 진심 빡친다. 신성모독,1,"[b',', b'\x0e', b'\x9e', b'>', b'O', b'H', b'g...","{'keyword': ['진심 빡친다 신성모독', '신세계에 비교하는 진심', '비..."
3,9702804,킬링타임 최고다 열자채워,1,"[b'\xde', b'\x92', b'S', b'?', b'\x15', b'\x17...","{'keyword': ['킬링타임 최고다 열자채워', '킬링타임 최고다', '최고다..."
4,10097178,명불허전 선동영화 10자,0,"[b'\x86', b'j', b'\xb1', b'\xbe', b'&', b'\xbd...","{'keyword': ['명불허전 선동영화 10자', '명불허전 선동영화', '선동..."
5,9245590,결말이 마음에 든다. 소설을 먼저 읽었던 터라 결말이 마음에 든다. 세상의 모든 정...,1,"[b'\xa2', b'\x8d', b'\x12', b'>', b'\xeb', b'\...","{'keyword': ['읽었던 터라 결말이', '마음에 든다 세상의', '모든 정..."
6,4227156,기대했는데.. 시나리오는 ...어떻게 결말이 그렇지?..요즘 막장 드라마 영향인가..,0,"[b'\x05', b'3', b'\xd7', b'<', b'\xc6', b'\xeb...","{'keyword': ['기대했는데 시나리오는 어떻게', '시나리오는 어떻게 결말이..."
7,7313784,인기연예인 몇명 출현한다고 재미있는 영화냐 잡것들아긴급조치19호가 그렇게 만들고 망...,0,"[b'U', b'\xc9', b'\x00', b'>', b'\xf1', b'\x14...","{'keyword': ['영화냐 잡것들아긴급조치19호가 그렇게', '잡것들아긴급조치..."
8,3582981,좋았음!!,1,"[b'\xfc', b'\xbe', b'Z', b'?', b'\xf5', b'\x81...","{'keyword': ['좋았음'], 'score': [0.7752]}"
9,9906269,멜로와 스릴러를 이렇게 조합할수 있다니 ...,1,"[b'G', b'u', b'\r', b'\xbe', b'\x8c', b'\xec',...","{'keyword': ['멜로와 스릴러를 이렇게', '스릴러를 이렇게 조합할수', ..."


In [12]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        use_stopwords=True
        )
    AS
    SELECT *
    FROM nsmc_test_convert
    LIMIT 10
    )

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[봐야하는 영화인, 한번 봐야하는, 영화인 같네요, 같네요 ㅎㅎ, 한번]","[0.6659, 0.6458, 0.627, 0.5636, 0.4059]"
1,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[럽라인은작가빙의된니끼한조연과 여주는남자갖고논질나쁜여자, 권선징악은안드로메다로 럽라...","[0.9426, 0.9371, 0.8929, 0.8422, 0.7508]"
2,유쾌하고 신나는 스피드 코미디.,1,"[유쾌하고 신나는, 신나는 스피드, 스피드 코미디, 신나는, 유쾌하고]","[0.796, 0.7637, 0.7328, 0.675, 0.6482]"
3,정말 따뜻했던 드라마ㅠㅠ,1,"[정말 따뜻했던, 따뜻했던 드라마ㅠㅠ, 따뜻했던, 정말, 드라마ㅠㅠ]","[0.9032, 0.8929, 0.7773, 0.6903, 0.5206]"
4,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[솔직히 배우들의, 영화자체만을 놓고보면, 특색없고 너무, 뻔한 전개, 내세우는게 ...","[0.5393, 0.5271, 0.4929, 0.4555, 0.3465]"
5,느와르에 멜로에 신파극까지······. 짬뽕이 아니라 꿀꿀이죽,0,"[신파극까지 짬뽕이, 짬뽕이 아니라, 느와르에, 멜로에, 아니라 꿀꿀이죽]","[0.616, 0.4833, 0.4357, 0.4176, 0.3915]"
6,다시 보고 싶네요~,1,"[보고 싶네요, 다시 보고, 다시, 보고, 싶네요]","[0.7777, 0.7486, 0.6543, 0.4931, 0.4916]"
7,짱이야... 진짜 극장을 나오는데 멋진 소설을 한권 읽은 듯한 기분을 느낌. 재미는...,1,"[ㅠㅠㅠ 꼭보시길, 감동까지 철철, 짱이야 진짜, 듯한 기분을, 소설을 한권]","[0.5705, 0.569, 0.5334, 0.3728, 0.3601]"
8,유일하게 아들과 같이볼수있는 만화영화,1,"[같이볼수있는 만화영화, 아들과 같이볼수있는, 유일하게 아들과, 만화영화, 유일하게]","[0.7885, 0.7598, 0.6182, 0.6019, 0.5266]"
9,남자라면 봐라 꼭봐라 두번봐라 세번봐라 계속봐라,1,"[두번봐라 세번봐라, 세번봐라 계속봐라, 꼭봐라 두번봐라, 봐라 꼭봐라, 남자라면 봐라]","[0.8524, 0.8512, 0.8319, 0.702, 0.5616]"


In [13]:
%%thanosql
SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, json_array_elements(keyword -> 'score') AS score
FROM (
    SEARCH KEYWORD 
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        use_stopwords=True,
        threshold=0.5
        )
    AS
    SELECT *
    FROM nsmc_test_convert
    LIMIT 10
    )

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,봐야하는 영화인,0.6659
1,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,한번 봐야하는,0.6458
2,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,영화인 같네요,0.6270
3,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,같네요 ㅎㅎ,0.5636
4,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,럽라인은작가빙의된니끼한조연과 여주는남자갖고논질나쁜여자,0.9426
5,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,권선징악은안드로메다로 럽라인은작가빙의된니끼한조연과,0.9371
6,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,럽라인은작가빙의된니끼한조연과,0.8929
7,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,여주는남자갖고논질나쁜여자,0.8422
8,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,권선징악은안드로메다로,0.7508
9,유쾌하고 신나는 스피드 코미디.,1,유쾌하고 신나는,0.7960


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
            <ul>
                <li>"lang": 사용할 언어 (str, optional, 'ko'|'en', default: 'ko')</li>
                <li>"text_col": 텍스트가 있는 열 이름 (str, default: 'text')</li>
                <li>"ngram_range": 키워드의 최소 단어 수와 최대 단어수 예) [1, 3]. 대부분의 상황에서 키워드는 최대 단어 수에 맞춰 추출됨 (list[int, int], optional, default: [1, 2])</li>
                <li>"top_n": 추출할 키워드의 수, 유사도가 높은 순서대로 (int, optional, default: 5)</li>
                <li>"diversity": 추출될 키워드의 다양성. 높을 수록 기존에 추출된 키워드와 유사한 키워드는 다시 추출되지 않음 0 <= diversity <= 1 (float, optional, default: 0.5)</li>
                <li>"use_stopwords": 큰 의미가 없는 단어(불용어)를 제외할 지 여부 (bool, optional, True|False, default: True)</li>
                <li>"threshold": 추출할 키워드의 유사도 수치의 최소값 (float, optional, default: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __5. 두 방법 결합하여 사용__

In [14]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT document, label, score
    FROM (
        SEARCH TEXT
        USING nsmc_text_search_model
        OPTIONS (
            search_by='text',
            search_input='가볍게 볼 수 있는 코미디 영화',
            emb_col='convert_result',
            result_col='score',
            top_k=10
            )
        AS
        SELECT *
        FROM nsmc_test_convert
        )
    )

,document,label,score,keyword
0,순수하게 보면 참으로 재밌는 영화,1,0.708074,"{'keyword': ['순수하게 보면 참으로', '참으로 재밌는 영화', '참으로..."
1,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,0.689155,"{'keyword': ['레전드란 말이 어울리는', '안되는 영화중 하나', '레전..."
2,코미디로도 사람의 마음을 울릴수 있느 영화,1,0.675590,"{'keyword': ['울릴수 있느 영화', '사람의 마음을 울릴수', '마음을 ..."
3,감동적인 영화,1,0.673828,"{'keyword': ['감동적인 영화', '감동적인', '영화'], 'score'..."
4,오래됐지만 재미있는 영화,1,0.670266,"{'keyword': ['오래됐지만 재미있는 영화', '오래됐지만 재미있는', '오..."
5,간만에 본 한국영화 중 수작중의 수작,1,0.661964,"{'keyword': ['간만에 한국영화 수작중의', '한국영화 수작중의 수작', ..."
6,보고나면 기분 좋은 영화,1,0.655647,"{'keyword': ['보고나면 기분 좋은', '기분 좋은 영화', '좋은 영화'..."
7,극장에서 봐선 안될 영화였지.,0,0.650464,"{'keyword': ['봐선 안될 영화였지', '극장에서 봐선 안될', '안될 영..."
8,뭔가 있어보이는 듯 무게를 잡지만 결국 코미디 공포물,0,0.649996,"{'keyword': ['결국 코미디 공포물', '잡지만 결국 코미디', '있어보이..."
9,긴박한 영화만 좋아한다면 보지말구 영화 자체를 즐기는 사람이라면 감탄할 영화,1,0.643158,"{'keyword': ['영화만 좋아한다면 보지말구', '긴박한 영화만 좋아한다면'..."


In [15]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT document, label, score
        FROM (
            SEARCH TEXT
            USING nsmc_text_search_model
            OPTIONS (
                search_by='text',
                search_input='가볍게 볼 수 있는 코미디 영화',
                emb_col='convert_result',
                result_col='score',
                top_k=10
                )
            AS
            SELECT *
            FROM nsmc_test_convert
            )
        )
    )

,document,label,keywords,score
0,순수하게 보면 참으로 재밌는 영화,1,"[순수하게 보면 참으로, 참으로 재밌는 영화, 참으로 재밌는, 재밌는 영화, 보면]","[0.8469, 0.8151, 0.7329, 0.6429, 0.4257]"
1,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,"[레전드란 말이 어울리는, 안되는 영화중 하나, 레전드란, 영화중, 말이 어울리는 ...","[0.8029, 0.7411, 0.6903, 0.5925, 0.5421]"
2,코미디로도 사람의 마음을 울릴수 있느 영화,1,"[울릴수 있느 영화, 사람의 마음을 울릴수, 마음을 울릴수 있느, 코미디로도 사람의...","[0.7616, 0.7584, 0.7444, 0.7431, 0.3754]"
3,감동적인 영화,1,"[감동적인 영화, 감동적인, 영화]","[1.0, 0.9031, 0.6157]"
4,오래됐지만 재미있는 영화,1,"[오래됐지만 재미있는 영화, 오래됐지만 재미있는, 오래됐지만, 재미있는 영화, 영화]","[1.0, 0.8833, 0.8162, 0.7668, 0.5857]"
5,간만에 본 한국영화 중 수작중의 수작,1,"[간만에 한국영화 수작중의, 한국영화 수작중의 수작, 한국영화 수작중의, 수작중의 ...","[0.9008, 0.8923, 0.8578, 0.7515, 0.4676]"
6,보고나면 기분 좋은 영화,1,"[보고나면 기분 좋은, 기분 좋은 영화, 좋은 영화, 보고나면, 영화]","[0.9181, 0.8752, 0.7991, 0.7205, 0.5604]"
7,극장에서 봐선 안될 영화였지.,0,"[봐선 안될 영화였지, 극장에서 봐선 안될, 안될 영화였지, 영화였지, 극장에서]","[0.9213, 0.8463, 0.7928, 0.6556, 0.5567]"
8,뭔가 있어보이는 듯 무게를 잡지만 결국 코미디 공포물,0,"[결국 코미디 공포물, 잡지만 결국 코미디, 있어보이는 무게를 잡지만, 잡지만 결국...","[0.7573, 0.7252, 0.677, 0.5263, 0.3953]"
9,긴박한 영화만 좋아한다면 보지말구 영화 자체를 즐기는 사람이라면 감탄할 영화,1,"[영화만 좋아한다면 보지말구, 긴박한 영화만 좋아한다면, 사람이라면 감탄할 영화, ...","[0.7925, 0.7593, 0.6937, 0.6602, 0.5256]"


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __6. 튜토리얼을 마치며__

이번 튜토리얼에서는 nsmc 영화 리뷰 데이터 세트를 사용하여 텍스트 수치화와 수치화 결과를 바탕으로한 유사 텍스트 검색, 키워드 추출을 진행해 보았습니다. 이번 튜토리얼에서는 모델의 수치화 성능보다는 작동 위주의 설명으로 진행하였습니다. 나만의 텍스트 수치화 모델을 만들어 다양한 형태의 비정형 데이터 세트에 검색 기능을 추가하고 AutoML 기법을 이용한 나만의 모델을 배포할 수 있습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/1.5/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/1.5/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/1.5/ko/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>유사 텍스트 검색 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>